<a href="https://colab.research.google.com/github/PRKKILLER/cmpe255-spring21-1/blob/master/Assignment%202/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import fetch_lfw_people
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn import model_selection
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
faces = fetch_lfw_people(min_faces_per_person=60)
print('data loaded')


In [ ]:
target_names = faces.target_names
n, h, w = faces.images.shape

In [ ]:
photos = faces.data
labels = faces.target
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(photos, labels, test_size=0.25,random_state=42)

In [ ]:
pca = RandomizedPCA(n_components=150, svd_solver='randomized', whiten=True, random_state=42).fit(xtrain)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

In [ ]:
C = np.logspace(-1, 5, 4)
gammas = np.logspace(-3, 0, 4)
params = dict(svc__gamma=gammas, svc__C=C)
gsv = model_selection.GridSearchCV(model, params)

In [ ]:
gsv = gsv.fit(xtrain, ytrain)

In [ ]:
print(gsv.best_params_)
model = gsv.best_estimator_
ypred = model.predict(xtest)

In [ ]:
print(ypred)

In [ ]:
r_ind = np.random.choice(xtest.shape[0], size=24, replace=False)
photos = xtest[r_ind]
labels_actual = ytest[r_ind]
labels_pred_sample = ypred[r_ind]
names_pred = target_names[labels_pred_sample]
names_actual = target_names[labels_actual]
print(metrics.classification_report(ytest, ypred, target_names=target_names))

In [ ]:
#CONFUSIN MATRIX
confusion_matrix = metrics.confusion_matrix(names_pred,names_actual,labels=target_names)
print(confusion_matrix)

In [ ]:
#HEAT MAP
sns.heatmap(confusion_matrix, annot=True)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show(block=True)

In [ ]:
n_row=4
n_col=6
fig_title="Predictions"
fig = plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
for i in range(n_row * n_col):
  ax = fig.add_subplot(n_row, n_col, i + 1)
  ax.imshow(photos[i].reshape((h, w)), cmap=plt.cm.gray)
  fc = 'black'
  if names_pred[i]!=names_actual[i] :
    fc = 'red'
    title = "Predicted: "+names_pred[i]+"\nActual: "+names_actual[i]
    ax.set_title(names_pred[i], size=12,color=fc)
    plt.xticks(())
    plt.yticks(())
if fig_title: 
      fig.suptitle(fig_title+'\n', fontsize=20)